![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2F05+-+TensorFlow&file=05Tools+-+Prediction+-+Custom.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/05%20-%20TensorFlow/05Tools%20-%20Prediction%20-%20Custom.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2F05%2520-%2520TensorFlow%2F05Tools%2520-%2520Prediction%2520-%2520Custom.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/05%20-%20TensorFlow/05Tools%20-%20Prediction%20-%20Custom.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/05%20-%20TensorFlow/05Tools%20-%20Prediction%20-%20Custom.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# 05Tools: Prediction - Custom

Predictions from models created in the 05 series of notebooks.

This notebook is part of collection of examples that showcase many ways to serve models:
- Online:
    - Vertex AI Endpoints: Python, REST, CLI (gcloud): [05Tools - Prediction - Online.ipynb](./05Tools%20-%20Prediction%20-%20Online.ipynb)
    - Local with TensorFlow ModelServer: [05Tools - Prediction - Local.ipynb](./05Tools%20-%20Prediction%20-%20Local.ipynb)
    - (**THIS NOTEBOOK**) Custom: Build a custom container with TensorFlow ModelServer: [05Tools - Prediction - Custom.ipynb](./05Tools%20-%20Prediction%20-%20Custom.ipynb)
        - Remote Service with Cloud Run
        - Local Service with Docker Run
- Batch: [05Tools - Prediction - Batch.ipynb](./05Tools%20-%20Prediction%20-%20Batch.ipynb)
    - BigQuery ML Model Import
    - Vertex AI Batch Prediction Jobs

**Prerequisites:**
-  At least 1 of the notebooks in this series [05, 05a-05i]

**Conceptual Flow & Workflow**

<p align="center">
  <img alt="Conceptual Flow" src="../architectures/slides/05tools_pred_arch.png" width="45%">
&nbsp; &nbsp; &nbsp; &nbsp;
  <img alt="Workflow" src="../architectures/slides/05tools_pred_console.png" width="45%">
</p>

---
## Setup

### Package Installs (if needed)

This notebook uses the Python Clients for
- Google Service Usage
    - to enable APIs (Artifact Registry and Cloud Build)
- Artifact Registry
    - to create repositories for Docker containers
- Cloud Build
    - To build custom Docker containers

The cells below check to see if the required Python libraries are installed.  If any are not it will print a message to do the install with the associated pip command to use.  These installs must be completed before continuing this notebook.

In [1]:
try:
    import google.cloud.service_usage_v1
except ImportError:
    print('You need to pip install google-cloud-service-usage')
    !pip install google-cloud-service-usage -q

In [2]:
try:
    import google.cloud.artifactregistry_v1
except ImportError:
    print('You need to pip install google-cloud-artifact-registry')
    !pip install google-cloud-artifact-registry -q

In [3]:
try:
    import google.cloud.devtools.cloudbuild
except ImportError:
    print('You need to pip install google-cloud-build')
    !pip install google-cloud-build

### Environment

inputs:

In [7]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [8]:
REGION = 'us-central1'
EXPERIMENT = '05_predictions'
SERIES = '05'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud_prepped'

# Resources
DEPLOY_COMPUTE = 'n1-standard-4'
DEPLOY_IMAGE='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters

packages:

In [16]:
from google.cloud import aiplatform
from google.cloud import bigquery
from google.cloud import service_usage_v1
from google.cloud.devtools import cloudbuild_v1
from google.cloud import artifactregistry_v1
from google.cloud import storage

import tensorflow as tf
import requests
import json
import numpy as np

import multiprocessing

clients:

In [17]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bq = bigquery.Client()
gcs = storage.Client()
su_client = service_usage_v1.ServiceUsageClient()
ar_client = artifactregistry_v1.ArtifactRegistryClient()
cb_client = cloudbuild_v1.CloudBuildClient()

parameters:

In [18]:
BUCKET = PROJECT_ID
DIR = f"temp/{EXPERIMENT}"

environment:

In [19]:
!rm -rf {DIR}
!mkdir -p {DIR}

### Enable APIs

Using Cloud Build and Artifact Registry requires enabling these APIs for the Google Cloud Project.

Options for enabeling these.  In this notebook option 2 is used.
 1. Use the APIs & Services page in the console: https://console.cloud.google.com/apis
     - `+ Enable APIs and Services`
     - Search for Cloud Build and Enable
     - Search for Artifact Registry and Enable
 2. Use [Google Service Usage](https://cloud.google.com/service-usage/docs) API from Python
     - [Python Client For Service Usage](https://github.com/googleapis/python-service-usage)
     - [Python Client Library Documentation](https://cloud.google.com/python/docs/reference/serviceusage/latest)
     
The following code cells use the Service Usage Client to:
- get the state of the service
- if 'DISABLED':
    - Try enabling the service and return the state after trying
- if 'ENABLED' print the state for confirmation

#### Artifact Registry

In [20]:
artifactregistry = su_client.get_service(
    request = service_usage_v1.GetServiceRequest(
        name = f'projects/{PROJECT_ID}/services/artifactregistry.googleapis.com'
    )
).state.name


if artifactregistry == 'DISABLED':
    print(f'Artifact Registry is currently {artifactregistry} for project: {PROJECT_ID}')
    print(f'Trying to Enable...')
    operation = su_client.enable_service(
        request = service_usage_v1.EnableServiceRequest(
            name = f'projects/{PROJECT_ID}/services/artifactregistry.googleapis.com'
        )
    )
    response = operation.result()
    if response.service.state.name == 'ENABLED':
        print(f'Artifact Registry is now enabled for project: {PROJECT_ID}')
    else:
        print(response)
else:
    print(f'Artifact Registry already enabled for project: {PROJECT_ID}')

Artifact Registry already enabled for project: statmike-mlops-349915


#### Cloud Build

In [21]:
cloudbuild = su_client.get_service(
    request = service_usage_v1.GetServiceRequest(
        name = f'projects/{PROJECT_ID}/services/cloudbuild.googleapis.com'
    )
).state.name


if cloudbuild == 'DISABLED':
    print(f'Cloud Build is currently {cloudbuild} for project: {PROJECT_ID}')
    print(f'Trying to Enable...')
    operation = su_client.enable_service(
        request = service_usage_v1.EnableServiceRequest(
            name = f'projects/{PROJECT_ID}/services/cloudbuild.googleapis.com'
        )
    )
    response = operation.result()
    if response.service.state.name == 'ENABLED':
        print(f'Cloud Build is now enabled for project: {PROJECT_ID}')
    else:
        print(response)
else:
    print(f'Cloud Build already enabled for project: {PROJECT_ID}')

Cloud Build already enabled for project: statmike-mlops-349915


#### Cloud Run

In [167]:
cloudrun = su_client.get_service(
    request = service_usage_v1.GetServiceRequest(
        name = f'projects/{PROJECT_ID}/services/run.googleapis.com'
    )
).state.name


if cloudrun == 'DISABLED':
    print(f'Cloud Run is currently {cloudrun} for project: {PROJECT_ID}')
    print(f'Trying to Enable...')
    operation = su_client.enable_service(
        request = service_usage_v1.EnableServiceRequest(
            name = f'projects/{PROJECT_ID}/services/run.googleapis.com'
        )
    )
    response = operation.result()
    if response.service.state.name == 'ENABLED':
        print(f'Cloud Run is now enabled for project: {PROJECT_ID}')
    else:
        print(response)
else:
    print(f'Cloud Run already enabled for project: {PROJECT_ID}')

Cloud Run already enabled for project: statmike-mlops-349915


---
## Get a Model For Predictions
This project already has a model serving online predictions at a Vertex AI Endpoint.  This section will use the endpoint to retrieve the deployed model and get its information to use for batch prediction methods in this notebook.

### Get Endpoint

[Endpoint Properties and Methods](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Endpoint):

```python
endpoint
endpoint.display_name
endpoint.resource_name
endpoint.traffic_split
endpoint.list_models()
```

In [22]:
endpoints = aiplatform.Endpoint.list(filter = f"labels.series={SERIES}")
endpoint = endpoints[0]

In [23]:
print(f'Review the Endpoint in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/endpoints/{endpoint.name}?project={PROJECT_ID}')

Review the Endpoint in the Console:
https://console.cloud.google.com/vertex-ai/locations/us-central1/endpoints/1961322035766362112?project=statmike-mlops-349915


### Get Model at Endpoint
Using the model on the endpoint for the current series:

In [24]:
endpoint

resource name: projects/1026793852137/locations/us-central1/endpoints/1961322035766362112

In [25]:
#endpoint.list_models()[0]

In [26]:
model = aiplatform.Model(
    model_name = endpoint.list_models()[0].model+f'@{endpoint.list_models()[0].model_version_id}'
)

### Review Model Information

In [27]:
model.display_name

'05_05h'

In [28]:
model.resource_name

'projects/1026793852137/locations/us-central1/models/model_05_05h'

In [29]:
model.version_id

'1'

In [30]:
model.version_description

'run-20220927230247-6'

In [31]:
model.versioned_resource_name

'projects/1026793852137/locations/us-central1/models/model_05_05h@1'

In [32]:
model.supported_input_storage_formats

['jsonl', 'bigquery', 'csv', 'tf-record', 'tf-record-gzip', 'file-list']

In [33]:
model.name

'model_05_05h'

In [34]:
model.uri

'gs://statmike-mlops-349915/05/05h/models/20220927230247/6/model'

In [35]:
print(f'Review the model in the Vertex AI Model Registry:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/models/{model.name}/versions/{model.version_id}/properties?project={PROJECT_ID}')

Review the model in the Vertex AI Model Registry:
https://console.cloud.google.com/vertex-ai/locations/us-central1/models/model_05_05h/versions/1/properties?project=statmike-mlops-349915


#### Review Model Information Using the `aiplatform_v1` Model Client
It may also be helpful to try the [ModelServiceClient](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.services.model_service.ModelServiceClient) in version 1 of the client to review the model attributes.  Here is example code for trying this.

Curious about client versions and layers?  Check out this tip document [aiplatform_notes.md](../Tips/aiplatform_notes.md).

In [36]:
from google.cloud import aiplatform_v1

client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
ModelClientv1 = aiplatform_v1.ModelServiceClient(client_options = client_options)

ModelClientv1.get_model(
    name = model.versioned_resource_name
)

name: "projects/1026793852137/locations/us-central1/models/model_05_05h@1"
display_name: "05_05h"
predict_schemata {
}
metadata {
}
container_spec {
  image_uri: "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest"
}
supported_deployment_resources_types: DEDICATED_RESOURCES
supported_deployment_resources_types: SHARED_RESOURCES
supported_input_storage_formats: "jsonl"
supported_input_storage_formats: "bigquery"
supported_input_storage_formats: "csv"
supported_input_storage_formats: "tf-record"
supported_input_storage_formats: "tf-record-gzip"
supported_input_storage_formats: "file-list"
supported_output_storage_formats: "jsonl"
supported_output_storage_formats: "bigquery"
create_time {
  seconds: 1664323764
  nanos: 618427000
}
update_time {
  seconds: 1664323768
  nanos: 500624000
}
deployed_models {
  endpoint: "projects/1026793852137/locations/us-central1/endpoints/1961322035766362112"
  deployed_model_id: "6805735083375329280"
}
etag: "AMEw9yOIT4zwTOutKsVTaTYRSmfpynv-164Qtot

---
## Retrieve Records For Prediction

In [37]:
n = 1000
pred = bq.query(query = f"SELECT * FROM {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE} WHERE splits='TEST' LIMIT {n}").to_dataframe()

In [38]:
pred.head(4)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,35337,1.092844,-0.013230,1.359829,2.731537,-0.707357,0.873837,-0.796130,0.437707,0.396770,...,-0.167647,0.027557,0.592115,0.219695,0.036970,0.010984,0.0,0,a1b10547-d270-48c0-b902-7a0f735dadc7,TEST
1,60481,1.238973,0.035226,0.063003,0.641406,-0.260893,-0.580097,0.049938,-0.034733,0.405932,...,-0.057718,0.104983,0.537987,0.589563,-0.046207,-0.006212,0.0,0,814c62c8-ade4-47d5-bf83-313b0aafdee5,TEST
2,139587,1.870539,0.211079,0.224457,3.889486,-0.380177,0.249799,-0.577133,0.179189,-0.120462,...,0.180776,-0.060226,-0.228979,0.080827,0.009868,-0.036997,0.0,0,d08a1bfa-85c5-4f1b-9537-1c5a93e6afd0,TEST
3,162908,-3.368339,-1.980442,0.153645,-0.159795,3.847169,-3.516873,-1.209398,-0.292122,0.760543,...,-1.171627,0.214333,-0.159652,-0.060883,1.294977,0.120503,0.0,0,802f3307-8e5a-4475-b795-5d5d8d7d0120,TEST


Remove columns not included as features in the model:

In [39]:
newobs = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET, 'splits'])]].to_dict(orient='records')
#newobs[0]

In [40]:
len(newobs)

1000

In [41]:
newobs[0]

{'Time': 35337,
 'V1': 1.0928441854981998,
 'V2': -0.0132303486713432,
 'V3': 1.35982868199426,
 'V4': 2.7315370965921004,
 'V5': -0.707357349219652,
 'V6': 0.8738370029866129,
 'V7': -0.7961301510622031,
 'V8': 0.437706509544851,
 'V9': 0.39676985012996396,
 'V10': 0.587438102569443,
 'V11': -0.14979756231827498,
 'V12': 0.29514781622888103,
 'V13': -1.30382621882143,
 'V14': -0.31782283120234495,
 'V15': -2.03673231037199,
 'V16': 0.376090905274179,
 'V17': -0.30040350116459497,
 'V18': 0.433799615590844,
 'V19': -0.145082264348681,
 'V20': -0.240427548108996,
 'V21': 0.0376030733329398,
 'V22': 0.38002620963091405,
 'V23': -0.16764742731151097,
 'V24': 0.0275573495476881,
 'V25': 0.59211469704354,
 'V26': 0.219695164116351,
 'V27': 0.0369695108704894,
 'V28': 0.010984441006191,
 'Amount': 0.0}

---
## Create Custom Container For Model Serving

Cloud Build will be used to create a custom container with a [tensorflow/serving repository](https://hub.docker.com/r/tensorflow/serving/tags/) image that includes a model.  The resulting image will be stored in Artifact Registry.

To learn more about creating custom containers for ML workflows visit the tips notebook [Python Custom Containers](../Tips/Python%20Custom%20Container.ipynb).

#### Store Resources in Cloud Storage

In [72]:
bucket = gcs.lookup_bucket(PROJECT_ID)
SOURCEPATH = f'{SERIES}/{EXPERIMENT}/serving'

**Note** In this series of notebooks `05` used the local notebook to train with TensorFlow while the notebooks `05a-i` use a pre-built container with a possibly different version. Knowing these versions may be important to pull the correct image from the tensorflow/serving repository of images.  The example below pulls the imaged tagged `2.7.4` to align with the version of TensorFlow used to train modles in notebooks `05a-i`.  

In [73]:
serving_version = '2.7.4'

#### Create the Dockerfile
A basic dockerfile thats take the base image and copies the model in and define the startup commands to run.

In [140]:
dockerfile = f"""
FROM tensorflow/serving:{serving_version}
#ENTRYPOINT [“/usr/bin/env”]
ENV MODEL_NAME={SERIES}
ENV PORT=8501
#COPY model/* /models/{SERIES}/1/
COPY model /models/{SERIES}/1
CMD tensorflow_model_server --port8500 --rest_api_port=$PORT --model_base_path=/models/{SERIES} --model_name=$MODEL_NAME
"""

In [141]:
blob = bucket.blob(f'{SOURCEPATH}/Dockerfile')
blob.upload_from_string(dockerfile)

#### Setup Artifact Registry

Artifact registry organizes artifacts with repositories.  Each repository contains packages and is designated to hold a partifcular format of package: Docker images, Python Packages and [others](https://cloud.google.com/artifact-registry/docs/supported-formats#package).

##### List Repositories

This may be empty if no repositories have been created for this project

In [142]:
for repo in ar_client.list_repositories(parent = f'projects/{PROJECT_ID}/locations/{REGION}'):
    print(repo.name)

projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915
projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915-docker
projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915-python


#### Create Docker Image Repository

Create an Artifact Registry Repository to hold Docker Images created by this notebook.  First, check to see if it is already created by a previous run and retrieve it if it has.  Otherwise, create!

In [143]:
docker_repo = None
for repo in ar_client.list_repositories(parent = f'projects/{PROJECT_ID}/locations/{REGION}'):
    if f'{PROJECT_ID}-docker' in repo.name:
        docker_repo = repo
        print(f'Retrieved existing repo: {docker_repo.name}')

if not docker_repo:
    operation = ar_client.create_repository(
        request = artifactregistry_v1.CreateRepositoryRequest(
            parent = f'projects/{PROJECT_ID}/locations/{REGION}',
            repository_id = f'{PROJECT_ID}-docker',
            repository = artifactregistry_v1.Repository(
                description = f'A repository for the {EXPERIMENT} experiment that holds docker images.',
                name = f'{PROJECT_ID}-docker',
                format_ = artifactregistry_v1.Repository.Format.DOCKER,
                labels = {'series': SERIES, 'experiment': EXPERIMENT}
            )
        )
    )
    print('Creating Repository ...')
    docker_repo = operation.result()
    print(f'Completed creating repo: {docker_repo.name}')

Retrieved existing repo: projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915-docker


In [144]:
docker_repo.name, docker_repo.format_.name

('projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915-docker',
 'DOCKER')

In [145]:
REPOSITORY = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{docker_repo.name.split('/')[-1]}"

#### Build Custom Container
Use the Cloud Build client to construct and run the build instructions.  Here the files collected in GCS are copied to the build instance, then the Docker build in run in the folder with the `Dockerfile`.  The resulting image is pushed to Artifact Registry (setup above).

In [146]:
model.uri

'gs://statmike-mlops-349915/05/05h/models/20220927230247/6/model'

In [147]:
# setup the build config with empty list of steps - these will be added sequentially
build = cloudbuild_v1.Build(
    steps = []
)
# copy the model - will create folder /model/<contents, like .pb>
build.steps.append(
    {
        'name': 'gcr.io/cloud-builders/gsutil',
        'args': ['cp', '-r', f'{model.uri}', '/workspace']
    }
)
# retrieve the Dockerfile
build.steps.append(
    {
        'name': 'gcr.io/cloud-builders/gsutil',
        'args': ['cp', '-r', f'gs://{PROJECT_ID}/{SOURCEPATH}/Dockerfile', '/workspace']
    }
)
# docker build
build.steps.append(
    {
        'name': 'gcr.io/cloud-builders/docker',
        'args': ['build', '-t', f'{REPOSITORY}/{EXPERIMENT}', '/workspace']
    }    
)
# docker push
build.images = [f"{REPOSITORY}/{EXPERIMENT}"]

In [148]:
build

steps {
  name: "gcr.io/cloud-builders/gsutil"
  args: "cp"
  args: "-r"
  args: "gs://statmike-mlops-349915/05/05h/models/20220927230247/6/model"
  args: "/workspace"
}
steps {
  name: "gcr.io/cloud-builders/gsutil"
  args: "cp"
  args: "-r"
  args: "gs://statmike-mlops-349915/05/05_predictions/serving/Dockerfile"
  args: "/workspace"
}
steps {
  name: "gcr.io/cloud-builders/docker"
  args: "build"
  args: "-t"
  args: "us-central1-docker.pkg.dev/statmike-mlops-349915/statmike-mlops-349915-docker/05_predictions"
  args: "/workspace"
}
images: "us-central1-docker.pkg.dev/statmike-mlops-349915/statmike-mlops-349915-docker/05_predictions"

In [149]:
operation = cb_client.create_build(
    project_id = PROJECT_ID,
    build = build
)

In [150]:
response = operation.result()
response.status, response.artifacts

(<Status.SUCCESS: 3>,
 images: "us-central1-docker.pkg.dev/statmike-mlops-349915/statmike-mlops-349915-docker/05_predictions")

In [151]:
print(f"Review the Custom Container with Artifact Registry in the Google Cloud Console:\nhttps://console.cloud.google.com/artifacts/docker/{PROJECT_ID}/{REGION}/{PROJECT_ID}-docker?project={PROJECT_ID}")

Review the Custom Container with Artifact Registry in the Google Cloud Console:
https://console.cloud.google.com/artifacts/docker/statmike-mlops-349915/us-central1/statmike-mlops-349915-docker?project=statmike-mlops-349915


---
## Cloud Run: TensorFlow ModelSever

Cloud Run will be used to host our custom container and handle prediction requests.

### Deploy as Cloud Run Service
This demonstration creates an open service allowing all traffic.  Review documentation for [Cloud Run](https://cloud.google.com/run/docs/overview/what-is-cloud-run) and the [CLOUD SKD CLI sections](https://cloud.google.com/sdk/gcloud/reference/run) for `gcloud run`.


If you have a policy inforced for 'Domain Restricted Sharing' then it may need adjusting for the project to allow this.  This should be done with care and you may wish to only accept authenticated or internal traffic.  Review options for authentication [here](https://cloud.google.com/run/docs/authenticating/overview).

Updated Org Policy:
- Logged in as Admin
- IAM > Organization Policies
    - Changed to Project (not org level)
    - Filter 'Domain Restricted Sharing'
    - Select and Edit
        - Applies to = Customize
        - Policy enforcement = Replace
        - Rules = Allow all
    - Save

View the Cloud Run Console for this project:

In [169]:
print(f'https://console.cloud.google.com/run?project={PROJECT_ID}')

https://console.cloud.google.com/run?project=statmike-mlops-349915


Create the Cloud Run Service:

In [171]:
!gcloud run deploy endpoint-$SERIES --image=$REPOSITORY/$EXPERIMENT --port=8501 --region=$REGION --platform=managed --allow-unauthenticated --no-user-output-enabled

Deploying new service...                                                       
  . Creating Revision...                                                       
  . Routing traffic...                                                         
  . Setting IAM Policy...                                                      


In [172]:
!gcloud run services list

   SERVICE      REGION       URL                                          LAST DEPLOYED BY                                     LAST DEPLOYED AT
✔  endpoint-05  us-central1  https://endpoint-05-urlxi72dpa-uc.a.run.app  1026793852137-compute@developer.gserviceaccount.com  2022-09-30T15:51:30.142570Z


In [173]:
services = !gcloud run services list --format="json" --filter=SERVICE:endpoint-$SERIES
services = json.loads("".join(services))[0]
services['status']['url']

'https://endpoint-05-urlxi72dpa-uc.a.run.app'

If you had to adjust a `Domain Restricted Sharing` policy after deployment then this command can update the service to allow all traffic:

In [225]:
#!gcloud run services add-iam-policy-binding --region=us-central1 --member='allUsers' --role=roles/run.invoker endpoint-$SERIES

### Get Predictions Using Cloud Run Service

Prepare in instance (observation) for prediction request:

In [174]:
tryob_json = json.dumps({"instances": [newobs[0]]})
tryob_json

'{"instances": [{"Time": 35337, "V1": 1.0928441854981998, "V2": -0.0132303486713432, "V3": 1.35982868199426, "V4": 2.7315370965921004, "V5": -0.707357349219652, "V6": 0.8738370029866129, "V7": -0.7961301510622031, "V8": 0.437706509544851, "V9": 0.39676985012996396, "V10": 0.587438102569443, "V11": -0.14979756231827498, "V12": 0.29514781622888103, "V13": -1.30382621882143, "V14": -0.31782283120234495, "V15": -2.03673231037199, "V16": 0.376090905274179, "V17": -0.30040350116459497, "V18": 0.433799615590844, "V19": -0.145082264348681, "V20": -0.240427548108996, "V21": 0.0376030733329398, "V22": 0.38002620963091405, "V23": -0.16764742731151097, "V24": 0.0275573495476881, "V25": 0.59211469704354, "V26": 0.219695164116351, "V27": 0.0369695108704894, "V28": 0.010984441006191, "Amount": 0.0}]}'

Make prediction request:

In [175]:
json_response = requests.post(f"{services['status']['url']}/v1/models/{SERIES}:predict", data=tryob_json, headers={"content-type": "application/json"})

In [176]:
json_response

<Response [200]>

In [177]:
print(json_response.text)

{
    "predictions": [[0.999359429, 0.000640570885]
    ]
}


In [178]:
predictions = json.loads(json_response.text)['predictions']
predictions

[[0.999359429, 0.000640570885]]

In [179]:
np.argmax(predictions[0])

0

### Remove Service
This command will remove the service.

Alternatively, you could adjust the service to not accept traffic.  Cloud Run will scale down to zero - or only charge when CPU is used (startup, shutdown, and receiving requests) unless `--no-cpu-throttling` is used ([documentation](https://cloud.google.com/run/docs/configuring/cpu-allocation#setting)).

In [180]:
!gcloud run services delete --region=us-central1 --quiet endpoint-$SERIES

Deleting [endpoint-05]...done.                                                 
Deleted service [endpoint-05].


---
## Local (to Notebook): TensorFlow Model Server

The local notebook environment may have docker installed. For example, Vertex AI Workbench user-managed instances do have docker installed.  In this case the custom container in Artfact Registry created above can be pulled (`docker pull`) and run (`docker run`) here to serve prediction requests from the local compute instance.  This can be great for testing, troubleshooting, or doing local experiments. It can also potentially be helpful when you need to try various prediction requests scenarios for interpretation/explainability like is done in these notebooks:
- [05Tools - Interpretability with LIT.ipynb](./05Tools%20-%20Interpretability%20with%20LIT.ipynb)
- [05Tools - Interpretability with WIT.ipynb](./05Tools%20-%20Interpretability%20with%20WIT.ipynb)

### Pull the custom serving image locally

In [152]:
!docker pull $REPOSITORY/$EXPERIMENT

Using default tag: latest
latest: Pulling from statmike-mlops-349915/statmike-mlops-349915-docker/05_predictions

8a513d66: Already exists 
803b7168: Already exists 
227c53ba: Already exists 
6b95623a: Already exists 
8a441e9f: Already exists 
Digest: sha256:dcb92088379de27cd1d32a8981eb2b2c4998a52af77a00341366d654bf1ea69b
Status: Downloaded newer image for us-central1-docker.pkg.dev/statmike-mlops-349915/statmike-mlops-349915-docker/05_predictions:latest
us-central1-docker.pkg.dev/statmike-mlops-349915/statmike-mlops-349915-docker/05_predictions:latest


### Run the serving image locally

The container is going to be run with commands in this notebook.  In order to run the serving while not tying up further exectutions in this notebook, a subprocess will be launched using `multiprocessing`. To learn more about multiprocessing and running tasks from Python in parallel visit the tips notebook [Python Multiprocessing](../Tips/Python%20Multiprocessing.ipynb).

First, build the syntax of the `docker run` command.

In [153]:
command = f'''docker run -t --rm -p 8501:8501 {REPOSITORY}/{EXPERIMENT}'''
print(command)

docker run -t --rm -p 8501:8501 us-central1-docker.pkg.dev/statmike-mlops-349915/statmike-mlops-349915-docker/05_predictions


Run the command in a subprocess at the local folder of this notebook - use multiprocess.Process():

In [154]:
def docker_runner():
    !{command}
    #!docker run -t -p 8501:8501 -v "/$(pwd)/temp/05_predictions/model:/models/05/01" -e MODEL_NAME=05 tensorflow/serving:2.7.4

def main():
    p = multiprocessing.Process(target=docker_runner)
    p.start()
    return p
    
p = main()

unknown argument: /bin/sh
usage: tensorflow_model_server
Flags:
	--port=8500                      	int32	TCP port to listen on for gRPC/HTTP API. Disabled if port set to zero.
	--grpc_socket_path=""            	string	If non-empty, listen to a UNIX socket for gRPC API on the given path. Can be either relative or absolute path.
	--rest_api_port=0                	int32	Port to listen on for HTTP/REST API. If set to zero HTTP/REST API will not be exported. This port must be different than the one specified in --port.
	--rest_api_num_threads=16        	int32	Number of threads for HTTP/REST API processing. If not set, will be auto set based on number of CPUs.
	--rest_api_timeout_in_ms=30000   	int32	Timeout for HTTP/REST API calls.
	--rest_api_enable_cors_support=false	bool	Enable CORS headers in response
	--enable_batching=false          	bool	enable batching
	--allow_version_labels_for_unavailable_models=false	bool	If true, allows assigning unused version labels to models that are not ava

### Get Predictions on Exposed Port

Prepare in instance (observation) for prediction request:

In [155]:
tryob_json = json.dumps({"instances": [newobs[0]]})
tryob_json

'{"instances": [{"Time": 35337, "V1": 1.0928441854981998, "V2": -0.0132303486713432, "V3": 1.35982868199426, "V4": 2.7315370965921004, "V5": -0.707357349219652, "V6": 0.8738370029866129, "V7": -0.7961301510622031, "V8": 0.437706509544851, "V9": 0.39676985012996396, "V10": 0.587438102569443, "V11": -0.14979756231827498, "V12": 0.29514781622888103, "V13": -1.30382621882143, "V14": -0.31782283120234495, "V15": -2.03673231037199, "V16": 0.376090905274179, "V17": -0.30040350116459497, "V18": 0.433799615590844, "V19": -0.145082264348681, "V20": -0.240427548108996, "V21": 0.0376030733329398, "V22": 0.38002620963091405, "V23": -0.16764742731151097, "V24": 0.0275573495476881, "V25": 0.59211469704354, "V26": 0.219695164116351, "V27": 0.0369695108704894, "V28": 0.010984441006191, "Amount": 0.0}]}'

Make prediction request:

In [156]:
json_response = requests.post(f'http://localhost:8501/v1/models/{SERIES}:predict', data=tryob_json, headers={"content-type": "application/json"})

In [157]:
json_response

<Response [200]>

In [158]:
print(json_response.text)

{
    "predictions": [[0.999359429, 0.000640570885]
    ]
}


In [159]:
predictions = json.loads(json_response.text)['predictions']
predictions

[[0.999359429, 0.000640570885]]

In [160]:
np.argmax(predictions[0])

0

### Shutdown TensorFlow Serving Container
There are two entities running: a subprocess called `p` and a docker container that was run by the subprocess.  It is not enough to just stop `p` but it might be enough to stop the container and then the subprocess will terminate due to completion.  The command below stop the subprocess `p` and then stop and remove the container.

In [161]:
p.terminate()

In [162]:
p.is_alive()

False

In [163]:
docker = !docker ps -a
docker

['CONTAINER ID   IMAGE                                                                                          COMMAND                  CREATED              STATUS              PORTS                              NAMES',
 '14f0a2c5442c   us-central1-docker.pkg.dev/statmike-mlops-349915/statmike-mlops-349915-docker/05_predictions   "/usr/bin/tf_serving…"   About a minute ago   Up About a minute   8500/tcp, 0.0.0.0:8501->8501/tcp   clever_stonebraker',
 'cfc6fa1ae606   gcr.io/inverting-proxy/agent                                                                   "/bin/sh -c \'/opt/bi…"   6 weeks ago          Up 3 days                                              proxy-agent']

In [164]:
for d in docker:
    if f'{REPOSITORY}/{EXPERIMENT}' in d:
        print(d.split()[-1])
        !docker stop {d.split()[-1]}
        !docker rm {d.split()[0]}

clever_stonebraker
clever_stonebraker
Error: No such container: 14f0a2c5442c


In [165]:
!docker ps -a

CONTAINER ID   IMAGE                          COMMAND                  CREATED       STATUS      PORTS     NAMES
cfc6fa1ae606   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   6 weeks ago   Up 3 days             proxy-agent
